In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 7 Linear Regression with TensorFlow using the California Housing Dataset
The goal of this exercise is to implement a linear regression model using TensorFlow to predict house prices based on the California Housing Dataset. The dataset contains various features such as average income, housing average age, and more. Your task is to build a linear regression model and evaluate its performance.

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Load the California Housing Dataset

In [ ]:
# Fetch the California Housing dataset
raw = fetch_california_housing()
X = pd.DataFrame(data=raw['data'], columns=raw['feature_names'])
y = pd.Series(raw['target'])


In [ ]:
X.describe()

Preprocess the data:

Normalize the features using the mean and standard deviation.

Split the dataset into training and testing sets (e.g., 80% for training, 20% for testing).

In [ ]:
# Normalize the features using mean and standard deviation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

Define the TensorFlow graph:

Create placeholders for the input features (X) and target variable (y).

Create variables for the model's weights (W) and bias (b).

Define the linear regression model using the equation: y_pred = X * W + b.

Define the loss function as the mean squared error between the predicted values and the true values.

Choose an optimizer (e.g., Gradient Descent) to minimize the loss function.

In [ ]:
# Convert the input data to float64
X_train = X_train.astype(np.float64)
y_train = y_train.astype(np.float64)
X_test = X_test.astype(np.float64)
y_test = y_test.astype(np.float64)

# Define the TensorFlow graph
input_size = X_train.shape[1]

X = tf.keras.Input(shape=(input_size,), name='X', dtype=tf.float64)
y = tf.keras.Input(shape=(), name='y', dtype=tf.float64)

W = tf.Variable(tf.random.normal([input_size, 1], dtype=tf.float64), name='weights')
b = tf.Variable(tf.random.normal([1], dtype=tf.float64), name='bias')

y_pred = tf.matmul(X, W) + b

# Define the loss function
loss = tf.reduce_mean(tf.square(y_pred - y))

# Choose an optimizer
optimizer = tf.optimizers.Adam(learning_rate=0.01)

Train the model:

Initialize TensorFlow session.

Initialize the model's variables.

Set the number of training epochs and the learning rate.

For each epoch, iterate through the training dataset and update the model's parameters using the optimizer.

Print the training loss at regular intervals.

In [ ]:
# Train the model
num_epochs = 100
batch_size = 32
num_batches = len(X_train) // batch_size

@tf.function
def train_step(X_batch, y_batch):
    with tf.GradientTape() as tape:
        y_pred_batch = tf.matmul(X_batch, W) + b
        batch_loss = tf.reduce_mean(tf.square(y_pred_batch - y_batch))
    
    gradients = tape.gradient(batch_loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))
    
    return batch_loss

for epoch in range(num_epochs):
    avg_loss = 0.0
    
    # Iterate over batches
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        
        batch_X = X_train[start_idx:end_idx]
        batch_y = y_train[start_idx:end_idx]
        
        # Run optimization
        batch_loss = train_step(batch_X, batch_y)
        
        avg_loss += batch_loss / num_batches
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}')



Evaluate the model:

Use the trained model to make predictions on the test dataset.

Calculate the mean squared error (MSE) between the predicted and true values.

Print the MSE as a measure of the model's performance.

In [ ]:
# Evaluate the model
y_pred_test = tf.matmul(X_test, W) + b
mse = tf.reduce_mean(tf.square(y_pred_test - y_test))
mse_value = mse.numpy()

print(f'\nMean Squared Error (MSE): {mse_value:.4f}')

A Mean Squared Error (MSE) value of 1.3140 indicates the average squared difference between the predicted and true values in your regression problem. In the case of house price prediction, a lower MSE value indicates that the predicted house prices are closer to the true prices in the dataset.

A low MSE value suggests that the linear regression model is performing well, and its predictions are relatively close to the true house prices. This is a positive outcome as it means the model is making accurate predictions.